In [ ]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

page_links = []

pages_links = ["https://www.bayut.eg/%D8%B9%D9%82%D8%A7%D8%B1%D8%A7%D8%AA-%D9%84%D9%84%D8%A8%D9%8A%D8%B9/%D9%85%D8%B5%D8%B1/?gclid=CjwKCAjwgfm3BhBeEiwAFfxrG2qLNEx54cQVYWVaZoYNx-L7KmJW3eC0_e5hqGBF6_MPJmiH_pSNnRoCrwYQAvD_BwE"]

for i in range(2, 2084):
    url = f"https://www.bayut.eg/صفحة-{i}/عقارات-للبيع/مصر/?gclid=CjwKCAjwgfm3BhBeEiwAFfxrG2qLNEx54cQVYWVaZoYNx-L7KmJW3eC0_e5hqGBF6_MPJmiH_pSNnRoCrwYQAvD_BwE"
    pages_links.append(url)

def scrape_page(url):
    try:
        response = requests.get(url )
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract property links
        ul_element = soup.find('ul', class_='e20beb46')
        if ul_element:
            li_elements = ul_element.find_all('li')
            for li in li_elements:
                a_tag = li.find('a', href=True)
                if a_tag:
                    full_url = "https://www.bayut.eg" + a_tag['href']
                    page_links.append(full_url)
    except Exception as e:
        print(f"Error scraping {url}: {e}")

with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(scrape_page, pages_links)




In [ ]:
len(page_links)

49860

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# Function to scrape a property page
def scrape_property_page(url, session):
    try:
        response = session.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract picture links
        pictures = [img['src'] for img in soup.find_all('img', class_='dfc8fd76')]

        # Extract price
        price = soup.find('span', class_='_2d107f6e')
        price = price.get_text(strip=True) if price else None

        # Extract location
        location = soup.find('div', class_='e4fd45f0')
        location = location.get_text(strip=True) if location else None

        # Extract number of rooms, bathrooms, and area
        room_info = soup.find('div', class_='_14f36d85')
        if room_info:
            info_spans = room_info.find_all('span', class_='_140e6903')
            rooms = info_spans[0].get_text(strip=True) if len(info_spans) > 0 else None
            bathrooms = info_spans[1].get_text(strip=True) if len(info_spans) > 1 else None
            area = info_spans[2].get_text(strip=True) if len(info_spans) > 2 else None
        else:
            rooms, bathrooms, area = None, None, None

        # Extract offer title
        offer = soup.find('h1', class_='d8b96890')
        offer = offer.get_text(strip=True) if offer else None

        # Extract description
        description = soup.find('span', class_='_3547dac9')
        description = description.get_text(strip=True) if description else None


        cpan_text = soup.find('span', class_='cpan _41163454')
        cpan_text = cpan_text.get_text(strip=True) if cpan_text else None

        # Extract details in ul
        ul_element = soup.find('ul', class_='_3dc8d08d')
        details = {}
        if ul_element:
            li_elements = ul_element.find_all('li')
            for idx, li in enumerate(li_elements):
                detail_label = li.get_text(strip=True)
                span = li.find('span', class_='_2fdf7fc5')
                detail_value = span.get_text(strip=True) if span else None
                if idx == 0:
                    details['نوع العقار'] = detail_value
                elif idx == 1:
                    details['نوع العرض'] = detail_value
                elif idx == 2:
                    details['الرقم المرجعي'] = detail_value
                elif idx == 3:
                    details['حالة البناء'] = detail_value
                elif idx == 4:
                    details['التأثيث'] = detail_value
                elif idx == 5:
                    details['تاريخ الإضافة'] = detail_value
                elif idx == 6:
                    details['الملكية العقارية'] = detail_value

        # Extract company name (Company providing the property)
        company = soup.find('span', class_='_02db0128')
        company_name = company.get_text(strip=True) if company else None

        # Extract real estate agent (وكيل العقار)
        agent = soup.find('span', class_='d8185451')
        agent_name = agent.get_text(strip=True) if agent else None

        # Return all the data
        return {
            'Unit Link': url,  # Adding the URL of the unit
            'Pictures': pictures,
            'Price': price,
            'Location': location,
            'Rooms': rooms,
            'forerunner':cpan_text,
            'Bathrooms': bathrooms,
            'Area': area,
            'Offer': offer,
            'Description': description,
            'نوع العقار': details.get('نوع العقار'),
            'نوع العرض': details.get('نوع العرض'),
            'الرقم المرجعي': details.get('الرقم المرجعي'),
            'حالة البناء': details.get('حالة البناء'),
            'التأثيث': details.get('التأثيث'),
            'تاريخ الإضافة': details.get('تاريخ الإضافة'),
            'الملكية العقارية': details.get('الملكية العقارية'),
            'Company': company_name,
            'Agent': agent_name  # Real estate agent
        }

    except Exception as e:
        print(f"Failed to scrape {url}: {e}")
        return None

# Function to scrape properties concurrently
def scrape_properties_concurrently(property_links):
    properties = []

    # Create a session object
    with requests.Session() as session:
        with ThreadPoolExecutor(max_workers=10) as executor:  # Adjust max_workers based on your CPU/memory capacity
            future_to_url = {executor.submit(scrape_property_page, url, session): url for url in property_links}
            for future in as_completed(future_to_url):
                url = future_to_url[future]
                try:
                    data = future.result()
                    if data:
                        properties.append(data)
                except Exception as e:
                    print(f"Error processing {url}: {e}")

    # Convert to a DataFrame
    df = pd.DataFrame(properties)
    return df

# Example usage (replace 'all_property_links' with your list of links)
property_data_df = scrape_properties_concurrently(page_links)

# Print the DataFrame
print(property_data_df)

# Save the DataFrame to a CSV file
property_data_df.to_csv("property_data.csv", index=False)


                                               Unit Link Pictures       Price  \
0      https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...       []  14,000,000   
1      https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...       []  13,000,000   
2      https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...       []   7,320,000   
3      https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...       []  14,632,930   
4      https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...       []   6,800,000   
...                                                  ...      ...         ...   
49855  https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...       []  14,000,000   
49856  https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...       []   8,000,000   
49857  https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...       []  22,000,000   
49858  https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...       []  11,700,000   
49859  https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...       []   1,800,000   

                           

In [ ]:


df=pd.DataFrame(property_data_df)
df.head(10)

,Unit Link,Pictures,Price,Location,Rooms,forerunner,Bathrooms,Area,Offer,Description,نوع العقار,نوع العرض,الرقم المرجعي,حالة البناء,التأثيث,تاريخ الإضافة,الملكية العقارية,Company,Agent
0,https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...,[],"14,000,000",جاردن ليكس، 6 أكتوبر، الجيزة,3 غرف,None,2 حمامات,154 متر مربع,شقة للبيع 154م في كمبوند جاردن ليكس Garden lak...,Garden Lakes - Hydeparkشقة مميزة للبيع في كمبو...,شقة,للبيع,بيوت - 6139-bV2TQl,قيد الإنشاء,5 أكتوبر 2024,أول سكن,None,Platinum Real Estate,Platinum
1,https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...,[],"13,000,000",كومباوند تاج سيتي، القاهرة الجديدة، القاهرة,3 غرف,None,2 حمامات,218 متر مربع,شقة بالروف للبيع بموقع متميز بكومباوند تاج سيت...,التفاصيلشقة بالروف للبيع بموقع متميز بكومباوند...,شقة,للبيع,بيوت - farahtajcity,قيد الإنشاء,غير المفروشة,2 أكتوبر 2024,أول سكن,Lacerta Real Estate,Farah
2,https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...,[],"7,320,000",جاردن ليكس، 6 أكتوبر، الجيزة,استوديو,None,1 حمام,72 متر مربع,استوديو للبيع بكمبوند جاردن ليكس فيو مميز باقل...,استوديو للبيع بكمبوند جاردن ليكس فيو مميز باقل...,شقة,للبيع,بيوت - HT,قيد الإنشاء,المفروشة,2 أكتوبر 2024,اعادة البيع,اووك ريل استيت,Oak real estate
3,https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...,[],"14,632,930",كومباوند زايد الجديدة، الشيخ زايد، الجيزة,3 غرف,None,5 حمامات,286 متر مربع,توين هاوس للبيع في كمبوند زايد الجديده,للبيع في V. Levels، الشيخ زايد الجديدةتفاصيل ا...,توين هاوس,للبيع,بيوت - 068,قيد الإنشاء,غير المفروشة,2 أكتوبر 2024,أول سكن,Royal Hills,Royal Hills
4,https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...,[],"6,800,000",ريف دو نيل، المعادي، القاهرة,1 غرفة,None,1 حمام,40 متر مربع,استلام فوري علي النيل مباشره بارده فندق عالمي ...,للتواصل مع المبيعات :عرض معلومات الاتصالwhatsa...,شقة,للبيع,بيوت - 5000329-CfzrMP,جاهز,غير المفروشة,2 أكتوبر 2024,أول سكن,Modern Cairo,Mohamed Abdallah
5,https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...,[],"12,000,000",لافيستا رأس الحكمة، الساحل الشمالي، مطروح,3 غرف,None,3 حمامات,150 متر مربع,شاليه على البحر للبيع بسعر مميز-لافيستا راس ال...,شاليه على البحر للبيع بسعر مميز -لافيستا راس ا...,شاليه,للبيع,بيوت - MO 27008,جاهز,غير المفروشة,2 أكتوبر 2024,اعادة البيع,يو ار اي,URE - Ultimate real Estate
6,https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...,[],"28,000,000",كومباوند ازار، القاهرة الجديدة، القاهرة,6 غرف,None,5 حمامات,355 متر مربع,للبيع توين هاوس داخل كمبوند ازار 1 التجمع الخا...,توين هاوس للبيع داخل كمبوند أزار 1 في التجمع ا...,توين هاوس,للبيع,بيوت - 320-4dAlBy,جاهز,غير المفروشة,2 أكتوبر 2024,اعادة البيع,Abrag Two,Ahmed Ibrahiem
7,https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...,[],"3,000,000",تاج سلطان، مدينة نصر، القاهرة,5 غرف,None,4 حمامات,238 متر مربع,فيلا لقطة للبيع بالسعر القديم في ارقي كمبوندات...,امتلك فيلا للبيع بكمبوند ( تاج سيتى Taj City )...,فیلا,للبيع,بيوت - 8279-ExljAT,جاهز,2 أكتوبر 2024,أول سكن,None,SFR,Ibrahim
8,https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...,[],"7,000,000",كومباوند بيفرلى هيلز، الشيخ زايد، الجيزة,3 غرف,None,3 حمامات,200 متر مربع,شقه متشطبه بجاردن استلام فوري في الجريا بيفرلي...,شقه متشطبه بجاردن استلام فوري في الجريا بيفرلي...,شقة,للبيع,بيوت - 9876-3uBaO6,جاهز,2 أكتوبر 2024,أول سكن,None,Property Hills Plus,Farah
9,https://www.bayut.eg/%D8%AA%D9%81%D8%A7%D8%B5%...,[],"3,005,255",سوان ليك ويست، الشيخ زايد، الجيزة,3 غرف,None,3 حمامات,204 متر مربع,شقة من حسن علام بلوكيشن مميز امام بالم هيلز ود...,شقة من حسن علام بلوكيشن مميز امام بالم هيلز ود...,شقة,للبيع,بيوت - 10380-IPKiOp,قيد الإنشاء,غير المفروشة,2 أكتوبر 2024,أول سكن,Property hills star,ezz atef
